# Problem 1:

Our cycle detection implementation comes from the given wikipedia article (https://en.wikipedia.org/wiki/Cycle_detection#Floyd's_Tortoise_and_Hare)

In [0]:
def floyd(f, x0):
  tortoise = f(x0)
  hare = f(tortoise)

  while tortoise != hare:
    tortoise = f(tortoise)
    hare = f(f(hare))
   
  mu = 0
  tortoise = x0
  while tortoise != hare:
      tortoise = f(tortoise)
      hare = f(hare)
      mu += 1

  lam = 1
  hare = f(tortoise)
  while tortoise != hare:
      hare = f(hare)
      lam += 1

  return lam, mu

#Testing

We can write some simple looping functions in order to test our cycle detection. We start by defining a wrapper which will print the output of our floyd's algorithm. 

In [0]:
def test_wrapper(f, x0):
  lam, mu = floyd(f, x0)

  print("Position of start of first cycle:")
  print(mu)
  print("Length of cycle:")
  print(lam)

  pattern = [f(x0)]
  for i in range(mu):
    pattern.append(f(pattern[-1]))
  for i in range(lam):
    pattern.append(f(pattern[-1]))

  pattern.append('...')

  print("Pattern up the first loop:")
  print(pattern)

We define $F_1$. It is clear to see that $F_1$ loops every 5 increments. If we pass $F_1$ into floyd's algorithm with $x_0 = 0$ we get the following output:

In [37]:
def f_1(x):
  return (x + 1) % 5

test_wrapper(f_1, 0)

Position of start of first cycle:
0
Length of cycle:
5
Pattern up the first loop:
[0, 1, 2, 3, 4, 0, '...']


Next we define $F_2$ in the following code block. We test it by passing in $x_0 = 15$. We can manually calculate the outputs of $F_2$ as follows:

[15, 30, 7, 14, 3, 6, 1, 2, 1, ...] 

The answer we expect is that the loop begins at position 6, and has a length of 2. 

In [36]:
def f_2(x):
  x = int(x)
  if x % 2 == 0:
    return int(x/4) if x/4 > 1 else 1
  if x % 2 == 1:
    return x*2

test_wrapper(f_2, 15)

Position of start of first cycle:
6
Length of cycle:
2
Pattern up the first loop:
[15, 30, 7, 14, 3, 6, 1, 2, 1, '...']


Finally, we will manually define a function that maps a set to itself. 

In [39]:
def f_3(x):
  switch = {
      0:1,
      1:5,
      2:3,
      3:6,
      4:2,
      5:4,
      6:4
  }

  return switch.get(x)

test_wrapper(f_3, 0)

Position of start of first cycle:
3
Length of cycle:
4
Pattern up the first loop:
[0, 1, 5, 4, 2, 3, 6, 4, '...']


Passed on the pattern, we can see that the values returned by the floyd algorihthm are correct. 

#Problem 2

We implement the method find_order, which calculates the multiplicative order of a number (https://en.wikipedia.org/wiki/Multiplicative_order). The definition is as follows; Given $a$ and $n$, find the smallest positive $k$ such that:

$a^k \equiv 1$ (mod n)

In other words, if $a^k$ mod n == 1, and $k$ is the smallest number which satisfies this expression, then $k$ is the order. 

In order to find $k$ using floyd's algorithm, we need to observe that once we find a value of k that satisfies the above expression, higher increments of k will just produce a cycle. Thus, our goal is to look for a cycle in the above expression and return the first value $k$ of that cycle. 

This is tricky however, if we simply define an iterative function g(k) = $(a^k)$ % N, with constants $a$ and $N$, the pattern we produce will not represent all powers of $a^k$. For example, based on the wikipedia article above, lets set $a = 4$ and $n = 7$. If we iterate over g we get:

g(1) = $4^1$ mod 7 = 4

g(4) = $4^4$ mod 7 = 4

g(4) = $4^4$ mod 7 = 4

...

This pattern does not tell us anything about the smallest value of $k$ which satisfies the expression. 

Instead, we actually want an iterative function that produces the set we would get by iterating k one value at a time:

g(1) = $4^1$ mod 7 = 4

g(2) = $4^2$ mod 7 = 2

g(3) = $4^3$ mod 7 = 1

g(4) = $4^4$ mod 7 = 4

The function that does this for us is the following:
g(x) = $a*k$ mod n

We see that this would give us the following:

g(1) = $4*1$ mod 7 = 4

g(4) = $4*4$ mod 7 = 2

g(2) = $4*2$ mod 7 = 1

g(1) = $4*1$ mod 7 = 4

At this point, floyd's would detect a cycle. 



In [113]:
def find_order(a, N):
  if np.gcd(a, int(N)) != 1:
    return -1

  #Setup a function f for use in the floyd function
  def f(k):
    return (a*k) % N

  lam, mu = floyd(f, 1)
  return lam

print("Length of cycle")
print(find_order(4,7))

Length of cycle
3


Here we note that the length of the cycle: 3, is the solution for $k$. 

#Tests

We run a comprehensive test by writing a more traditional find_order function. We then iterate $a, N$ over the range [1,100]. We test each value using the regular find_order method, and if an answer exists, we check to see if our find_order method returns the same answer. 

In [0]:
#find_order using the simple implementation to verify our results
def find_order_regular(a,N):
  for k in range(1,N):
    if (a**k) % N == 1:
      return k

  return -1

In [58]:
 def test_find_order():
  for i in range(100):
    for k in range(100):
      result = find_order_regular(i,k)
      if result != -1:
        #Check our answer against the real answer
        if find_order(i,k) != result:
          return False
  return True


print("Testing find_order")
print(test_find_order())

Testing find_order
True


#Problem 3

We implement this method in a straight-forward way based on the psuedo-code provided by the assignment. Based on the slides, we assume that N is not even, and that it is also not prime. 

The slides mention a step that is not in the psuedocode, which is to check if a and N are coprime. If they are not, then gcd(a,N) is a nontrivial factor of N. 

In [0]:
import numpy as np

def factor(N):

  while (1):
    # choose a uniformly at random in {2,...,N-1}
    a = np.random.randint(low=1, high=N, size=1)

    # compute the order r of a module N using the method from problem 2
    r = find_order(a, N)

    if r == -1:
      return np.gcd(a,N)

    # if r odd, chose different a
    if r % 2 == 1:
      continue

    # if r even, compute f = gcd(a**(r/2 -1), N))
    if r % 2 == 0:
      f = np.gcd(int(a**((r/2)-1)), int(N))

      # if f is a trivial factor, choose different a
      if f != 1 and f != N:
        return f

#Testing

We test our method by writing a test function that gets a factor for every number in the range [1,100]. We skip even and prime numbers in order to satisfy the conditions for N. We verify that the returned number is in fact a valid factor of the input. 

In [121]:
#A simple is prime function
def is_prime(N):
  for i in range(2,N):
    if N%i == 0:
      return False
  return True

def test_factor():
  for i in range(2,100):
    #Check that our input is valid
    if i % 2 == 0 or is_prime(i) or np.sqrt(i) % 1 == 0:
      continue

    #Get the factor
    result = factor(i)

    #Verify
    if i%result != 0:
      return False
  return True

print("Result of test_factor:")
print(test_factor())

Result of test_factor:
True
